In [1]:
import helper as hm
from helper import *

In [2]:
# magic
%matplotlib inline

# warnings
import warnings
warnings.filterwarnings(action='ignore')

# basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats

# other libraries
import pydotplus
from numpy import loadtxt
from xgboost import XGBClassifier
from IPython.display import Image  
from imblearn.ensemble import BalancedRandomForestClassifier 

# sci-kit learn libraries
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.feature_selection import rfe
from sklearn.feature_selection import RFECV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.utils import resample
from sklearn.externals.six import StringIO  

Using TensorFlow backend.


In [3]:
df = pd.read_csv('investments_VC.csv', encoding='ISO-8859-2')

In [4]:
df.dropna(inplace=True)

In [5]:
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [6]:
df['round_total'] = 0

In [7]:
for i in range(len(df)):
    df['round_total'][i]=np.sum([df.round_A[i],df.round_B[i],df.round_C[i],df.round_D[i],df.round_E[i],df.round_F[i],df.round_G[i],df.round_H[i]])

In [8]:
# df.drop(['round_A','round_B','round_C','round_D','round_E','round_F','round_G','round_H'],axis=1,inplace=True)

In [9]:
df['market']=df[' market ']
df.drop(['permalink','homepage_url','name','category_list',' funding_total_usd ',' market ','secondary_market'],axis=1,inplace=True)

In [10]:
df['founding_year_month'] = None

In [11]:
df.founded_month=pd.to_datetime(df.founded_month)

In [12]:
for i in range(len(df)):
    df.founded_month[i] = int(str(df.founded_month[i]).split()[0][5:7])

In [13]:
for i in range(len(df)):
    df.founding_year_month[i] = f'{df.founded_month[i]}/{int(df.founded_year[i])}'

In [14]:
df.drop('founded_at',axis=1,inplace=True)

In [15]:
status_dict = {'closed':0,'operating':1,'acquired':2}
df.status = df.status.map(lambda x: status_dict[x])

In [16]:
for i in range(len(df)):
    df.first_funding_at[i] = f'{str(df.first_funding_at[i]).split()[0][5:7]}/{str(df.first_funding_at[i]).split()[0][0:4]}'

In [17]:
for i in range(len(df)):
    df.last_funding_at[i] = f'{str(df.last_funding_at[i]).split()[0][5:7]}/{str(df.last_funding_at[i]).split()[0][0:4]}'

In [18]:
for i in range(len(df)):
    if df.product_crowdfunding[i] > 0:
        df.product_crowdfunding[i] = 1
    else:
        df.product_crowdfunding[i] = 0

In [19]:
for i in range(len(df)):
    if df.post_ipo_debt[i] > 0:
        df.post_ipo_debt[i] = 1
    else:
        df.post_ipo_debt[i] = 0

In [20]:
for i in range(len(df)):
    if df.post_ipo_equity[i] > 0:
        df.post_ipo_equity[i] = 1
    else:
        df.post_ipo_equity[i] = 0

In [21]:
for i in range(len(df)):
    if df.private_equity[i] > 0:
        df.private_equity[i] = 1
    else:
        df.private_equity[i] = 0

In [22]:
for i in range(len(df)):
    if df.grant[i] > 0:
        df.grant[i] = 1
    else:
        df.grant[i] = 0

In [23]:
for i in range(len(df)):
    if df.undisclosed[i] > 0:
        df.undisclosed[i] = 1
    else:
        df.undisclosed[i] = 0

In [24]:
for i in range(len(df)):
    if df.equity_crowdfunding[i] > 0:
        df.equity_crowdfunding[i] = 1
    else:
        df.equity_crowdfunding[i] = 0

In [25]:
for i in range(len(df)):
    if df.angel[i] > 0:
        df.angel[i] = 1
    else:
        df.angel[i] = 0

In [26]:
for i in range(len(df)):
    if df.venture[i] > 0:
        df.venture[i] = 1
    else:
        df.venture[i] = 0

In [ ]:
for i in range(len(df)):
    if df.convertible_note[i] > 0:
        df.convertible_note[i] = 1
    else:
        df.convertible_note[i] = 0

In [ ]:
for i in range(len(df)):
    if df.debt_financing[i] > 0:
        df.debt_financing[i] = 1
    else:
        df.debt_financing[i] = 0

In [ ]:
for feature in ['round_total','round_A','round_B','round_C','round_D','round_E','round_F','round_G','round_H']:
    for i in range(len(df)):
        if df[feature][i] > 0:
            df[feature][i] = 1
        else:
            df[feature][i] = 0

In [ ]:
for i in range(len(df)):
    if df.seed[i] > 0:
        df.seed[i] = 'seed_money'
    else:
        df.seed[i] = 'no_seed'

In [ ]:
cat_list = ['country_code','city','state_code','region','founded_month','founded_quarter','first_funding_at',
           'last_funding_at','seed','market','founding_year_month']

In [ ]:
df_cat=df[cat_list]

In [ ]:
for col in cat_list:
    df_cat = hm.create_encoding(col,df_cat,df,'status')

In [ ]:
df.drop(cat_list,axis=1,inplace=True)

In [ ]:
for col in df_cat:
    df[col]=df_cat[col]

In [ ]:
print(df.shape)
df.head()

In [ ]:
plt.tight_layout()
plt.figure(figsize=(15,8))
sns.heatmap(df.corr()>=0.7,cmap="YlGnBu")